In [2]:
import sys
from becquerel import Spectrum
import numpy as np
import matplotlib.pyplot as plt
import becquerel as bq
import scipy.integrate as integrate
import math as m


In [3]:
import PF
import ROI

In [4]:
source_isotopes = np.array(['K-40']);
source_energies = np.array([1460]);
source_activities = np.array([1]);
spectrum = r'C:\Users\Henricus\Downloads\UCB087_Wild_King_Salmon_3.Spe'
background =r'C:\Users\Henricus\Downloads\UCB096_Backgorund_2_13_17.Spe'
branching_ratio = np.array([1]);

#Call scripts
roi = ROI.ROI(Spectrum.from_file(spectrum), Spectrum.from_file(background), source_energies)
roi_result = roi.get_counts()
peakfit = PF.PF(source_energies,source_activities,spectrum,background,branching_ratio)
pf_result = peakfit.Efficiency()
print(roi_result, pf_result)

SpeFile: Reading file C:\Users\Henricus\Downloads\UCB087_Wild_King_Salmon_3.Spe
SpeFile: Reading file C:\Users\Henricus\Downloads\UCB096_Backgorund_2_13_17.Spe


c:\users\henricus\documents\github\becquerel\becquerel\core\spectrum.py:726: SpectrumWarning: Subtraction of counts-based specta, spectra have been converted to CPS
  'have been converted to CPS', SpectrumWarning)
c:\users\henricus\documents\github\becquerel\becquerel\core\spectrum.py:440: DeprecationWarning: energies_kev is deprecated and will be removed in a future release. Use bin_centers_kev instead.
  DeprecationWarning)


SpeFile: Reading file C:\Users\Henricus\Downloads\UCB087_Wild_King_Salmon_3.Spe
SpeFile: Reading file C:\Users\Henricus\Downloads\UCB096_Backgorund_2_13_17.Spe


AttributeError: module 'becquerel.core.fitting' has no attribute 'FitterGaussErfLine'